In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
from scipy.spatial import Voronoi
import vispy as vp


import tyssue

from tyssue.core.objects import Epithelium
from tyssue.geometry import bulk_geometry as bgeom
from tyssue.core.generation import (data_dicts, make_df,
                                    hexa_grid3d, from_3d_voronoi)


In [2]:
grid = hexa_grid3d(10, 10, 3)

In [3]:
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import seaborn as sns


fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(*grid.T, c='k', marker='o')

ax.set_xlabel('X Label')
ax.set_ylabel('Y Label')
ax.set_zlabel('Z Label')

plt.show()

/home/guillaume/anaconda/envs/py35/lib/python3.5/site-packages/matplotlib/__init__.py:872: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))


In [3]:
datasets = from_3d_voronoi(Voronoi(grid))
eptm = Epithelium('start3D', datasets)

In [4]:
from vispy import app, gloo, visuals, scene
from vispy.geometry import MeshData

In [8]:
gspecs = eptm.set_geom(bgeom)

In [9]:
bgeom.update_all(eptm)

In [10]:
eptm.cell_df.head()

,x,num_faces,vol,z,is_alive,y,area
cell,,,,,,,
0,-0.125,0,0.000000,0.6875,1,0.75,0
1,1.000,0,0.020833,0.5000,1,0.75,0
2,2.000,0,0.020833,0.5000,1,0.75,0
3,3.000,0,0.020833,0.5000,1,0.75,0
4,4.000,0,0.020833,0.5000,1,0.75,0


In [11]:
eptm.je_df.head()

,nz,dy,cell,trgt,dz,nx,ny,length,srce,dx,face,sub_area,sub_vol
je,,,,,,,,,,,,,
1565,0.3125,0.5,0,122,-0.25,0.15625,0.15625,0.559017,40,0.0,175,0.191366,0.000000
1563,0.2500,0.0,0,40,-0.50,0.12500,0.12500,1.118034,15,1.0,175,0.153093,0.000000
1561,0.3125,-0.5,0,15,0.00,0.15625,0.15625,0.707107,74,0.5,175,0.191366,0.000000
1559,0.3750,0.0,0,74,0.75,0.18750,0.18750,1.677051,122,-1.5,175,0.229640,0.000000
1607,0.1250,-0.5,1,40,0.25,-0.06250,0.06250,0.559017,122,0.0,180,0.076547,0.002604


In [14]:

vertices, faces, cell_mask = eptm.triangular_mesh(eptm.coords)

canvas = scene.SceneCanvas(keys='interactive', show=True)

grid = canvas.central_widget.add_grid()
view = grid.add_view(0, 1)
#view = canvas.central_widget.add_view()
view.camera =  'turntable'
view.camera.aspect = 1

cell_color = vp.color.ColorArray('#ff0000', alpha=0.5)
cell_color = cell_color.rgba.repeat(eptm.Ne, axis=0)

edge_color = vp.color.ColorArray('#00ff00', alpha=1)

view.bgcolor = vp.color.Color('#ffffff')


mesh = vp.scene.visuals.Mesh(vertices=vertices,
                             faces=faces,
                             face_colors=cell_color)

wire_pos = vertices[eptm.Nc:].copy()


wire = vp.scene.visuals.Line(pos=wire_pos, 
                             connect=faces[:, :2]-eptm.Nc,
                             color=edge_color,
                             width=20)

ccenters = vp.scene.visuals.Markers(pos=eptm.cell_df[eptm.coords].values)

view.add(mesh)
view.add(wire)
view.add(ccenters)
view.camera.set_range()
canvas.show()

#vp_widget = VispyWidget()
#vp_widget.set_canvas(canvas)



app.run()

0

In [15]:
eptm.je_df.head()

,nz,dy,cell,trgt,dz,nx,ny,length,srce,dx,face,sub_area,sub_vol
je,,,,,,,,,,,,,
1565,0.3125,0.5,0,122,-0.25,0.15625,0.15625,0.559017,40,0.0,175,0.191366,0.000000
1563,0.2500,0.0,0,40,-0.50,0.12500,0.12500,1.118034,15,1.0,175,0.153093,0.000000
1561,0.3125,-0.5,0,15,0.00,0.15625,0.15625,0.707107,74,0.5,175,0.191366,0.000000
1559,0.3750,0.0,0,74,0.75,0.18750,0.18750,1.677051,122,-1.5,175,0.229640,0.000000
1607,0.1250,-0.5,1,40,0.25,-0.06250,0.06250,0.559017,122,0.0,180,0.076547,0.002604
